In [1]:
import os
import sys
import pandas as pd
import io
import numpy as np
import boto3
import zipfile
import shutil

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)

# IMPORT WHEN PR #42 IS MERGED
'''
sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import (
    pull_zipped_csv, upload_csv_aws
)
'''

"\nsys.path.append(os.path.expanduser('../../'))\nfrom scripts.utils.file_helpers import (\n    pull_zipped_csv, upload_csv_aws\n)\n"

# 

In [2]:
def pull_csv_from_directory(bucket_name, directory, search_zipped=True):
    """
    Pulls CSV files from a specified directory in an S3 bucket.
    
    Parameters:
    - bucket_name (str): The name of the S3 bucket.
    - directory (str): The directory within the bucket to search for CSV files.
    - search_zipped (bool): If True, search for CSV files within zip files. If False, search for CSV files directly.
    """
    # Create an S3 client
    s3 = boto3.client('s3')

    # List objects in the specified directory
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=directory)

    # Check if objects were found
    if 'Contents' in response:
        # Iterate through each object found
        for obj in response['Contents']:
            # Get the key (filename) of the object
            key = obj['Key']
            
            # Check if the object is a .zip file
            if search_zipped and key.endswith('.zip'):
                # Download the zip file into memory
                zip_object = s3.get_object(Bucket=bucket_name, Key=key)
                zip_data = io.BytesIO(zip_object['Body'].read())
                
                # Open the zip file
                with zipfile.ZipFile(zip_data, 'r') as zip_ref:
                    # Iterate through each file in the zip
                    for file_name in zip_ref.namelist():
                        # Check if the file is a .csv file
                        if file_name.endswith('.csv'):
                            # Read the .csv file
                            with zip_ref.open(file_name) as csv_file:
                                # Convert the csv content to pandas DataFrame
                                df = pd.read_csv(csv_file)
                                # Save the DataFrame with a similar name as the .csv file
                                df_name = file_name[:-4]  # Remove .csv extension
                                df.to_csv(f"{df_name}.csv", index=False)
                                print(f"Saved DataFrame as '{df_name}.csv'")
                                # You can now manipulate df as needed
            elif not search_zipped and key.endswith('.csv'):
                # Directly download the CSV file
                csv_object = s3.get_object(Bucket=bucket_name, Key=key)
                csv_data = io.BytesIO(csv_object['Body'].read())
                # Convert the csv content to pandas DataFrame
                df = pd.read_csv(csv_data)
                # Save the DataFrame with a similar name as the .csv file
                df_name = key.split('/')[-1][:-4]  # Extract filename from key
                df.to_csv(f"{df_name}.csv", index=False)
                print(f"Saved DataFrame as '{df_name}.csv'")
                # You can now manipulate df as needed

    else:
        print("No objects found in the specified directory.")

def upload_csv_aws(file_name, bucket_name, directory):
    # Create an S3 client
    s3 = boto3.client('s3')
     # Save the file to AWS S3 using the client
    with open(file_name, 'rb') as data:
        s3.upload_fileobj(data, bucket_name, f"{directory}/{file_name}")
    print(f"{file_name} uploaded to AWS")

In [4]:
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/vulnerable_populations/american_community_survey/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=True)

Saved DataFrame as 'ACSDT5Y2022.B18105-Column-Metadata.csv'
Saved DataFrame as 'ambulatory_difficulty_B18105.csv'


C:\Users\jespi\AppData\Local\Temp\ipykernel_13520\2990477750.py:38: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Saved DataFrame as 'ACSDT5Y2022.B18104-Column-Metadata.csv'
Saved DataFrame as 'cognitive_difficulty_B18104.csv'


C:\Users\jespi\AppData\Local\Temp\ipykernel_13520\2990477750.py:38: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Saved DataFrame as 'ACSDP5Y2022.DP05-Column-Metadata.csv'


C:\Users\jespi\AppData\Local\Temp\ipykernel_13520\2990477750.py:38: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,232,240,248,312,326,360) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


Saved DataFrame as 'demographic_DP05.csv'
Saved DataFrame as 'ACSDT5Y2022.B09010-Column-Metadata.csv'
Saved DataFrame as 'financial_support_B09010.csv'
Saved DataFrame as 'ACSDT5Y2022.B27010-Column-Metadata.csv'
Saved DataFrame as 'health_insurance_B27010.csv'


C:\Users\jespi\AppData\Local\Temp\ipykernel_13520\2990477750.py:38: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [5]:
demographic_data = pd.read_csv('demographic_DP05.csv')
demographic_data.head(5)

C:\Users\jespi\AppData\Local\Temp\ipykernel_13520\2602433700.py:1: DtypeWarning: Columns (2,3,4,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,58,59,60,61,62,63,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,232,240,248,312,326,360) have mixed types. Specify dtype option on import or set low_memory=False.
  demographic_data = pd.read_csv('demographic_DP05.csv')


,GEO_ID,NAME,DP05_0001E,DP05_0001M,DP05_0002E,DP05_0002M,DP05_0003E,DP05_0003M,DP05_0004E,DP05_0004M,...,DP05_0087PM,DP05_0088PE,DP05_0088PM,DP05_0089PE,DP05_0089PM,DP05_0090PE,DP05_0090PM,DP05_0091PE,DP05_0091PM,Unnamed: 366
0,Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population...,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population...,Estimate!!SEX AND AGE!!Total population!!Sex r...,Margin of Error!!SEX AND AGE!!Total population...,...,Percent Margin of Error!!HISPANIC OR LATINO AN...,Percent!!Total housing units,Percent Margin of Error!!Total housing units,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...","Percent!!CITIZEN, VOTING AGE POPULATION!!Citiz...","Percent Margin of Error!!CITIZEN, VOTING AGE P...",NaN
1,1400000US06001400100,Census Tract 4001; Alameda County; California,3269,452,1621,339,1648,205,98.4,19.9,...,2.5,(X),(X),2351,(X),47.4,4.4,52.6,4.4,NaN
2,1400000US06001400200,Census Tract 4002; Alameda County; California,2147,201,1075,138,1072,129,100.3,16.3,...,3.4,(X),(X),1679,(X),49.4,4.5,50.6,4.5,NaN
3,1400000US06001400300,Census Tract 4003; Alameda County; California,5619,571,2801,504,2818,332,99.4,22.5,...,4.3,(X),(X),4414,(X),47.6,5.6,52.4,5.6,NaN
4,1400000US06001400400,Census Tract 4004; Alameda County; California,4278,598,1926,327,2352,363,81.9,13.5,...,1.8,(X),(X),3180,(X),46.9,4.0,53.1,4.0,NaN


In [7]:
demographic_data['Census_Tract'] = demographic_data['GEO_ID'].str[10:]
# Dropping first row which contains descriptions of row one columns
demographic_data = demographic_data.iloc[1:]
demographic_data[:2]

,GEO_ID,NAME,DP05_0001E,DP05_0001M,DP05_0002E,DP05_0002M,DP05_0003E,DP05_0003M,DP05_0004E,DP05_0004M,...,DP05_0088PE,DP05_0088PM,DP05_0089PE,DP05_0089PM,DP05_0090PE,DP05_0090PM,DP05_0091PE,DP05_0091PM,Unnamed: 366,Census_Tract
1,1400000US06001400100,Census Tract 4001; Alameda County; California,3269,452,1621,339,1648,205,98.4,19.9,...,(X),(X),2351,(X),47.4,4.4,52.6,4.4,NaN,6001400100
2,1400000US06001400200,Census Tract 4002; Alameda County; California,2147,201,1075,138,1072,129,100.3,16.3,...,(X),(X),1679,(X),49.4,4.5,50.6,4.5,NaN,6001400200


## Note - Come back and manually calculate percent over 65, column is wrong in data

In [10]:
# Renaming columns from dictionary code to definition
demographic_data = demographic_data.rename(columns={'DP05_0005PE': 'percent_total_pop_under_5'})
demographic_data = demographic_data.rename(columns={'DP05_0029PE': 'percent_total_pop_over_65'})
demographic_data = demographic_data.rename(columns={'DP05_0039PE': 'percent_total_pop_american_indian_alaska_native'})

In [11]:
# Isolating relevant columns to out data metric
cri_housing_vacancy_df = demographic_data[['GEO_ID', 'Census_Tract', 'percent_total_pop_under_5', 'percent_total_pop_over_65', 'percent_total_pop_american_indian_alaska_native']]


cri_housing_vacancy_df

# Saving metric df to .csv file
# cri_housing_vacancy_df.to_csv('built_metric_housing_vacancy_metric.csv')

,GEO_ID,Census_Tract,percent_total_pop_under_5,percent_total_pop_over_65,percent_total_pop_american_indian_alaska_native
1,1400000US06001400100,6001400100,4.1,884,0.0
2,1400000US06001400200,6001400200,7.9,553,0.4
3,1400000US06001400300,6001400300,2.3,916,0.5
4,1400000US06001400400,6001400400,7.5,550,0.5
5,1400000US06001400500,6001400500,4.0,649,0.1
...,...,...,...,...,...
9125,1400000US06115040902,6115040902,12.3,0,0.2
9126,1400000US06115041001,6115041001,6.9,1234,0.3
9127,1400000US06115041002,6115041002,2.6,1025,2.6
9128,1400000US06115041101,6115041101,2.7,478,1.1
